In [8]:
import re
import json
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import pandas as pd


In [9]:
class testNineentertain:

    def __init__(self):
        self.data = {}
        self.data['web'] = {}

    
    def remove_unuse_tag(self, bs : BeautifulSoup):
        unuse_tag = ['script', 'style ', 'noscript', 'head', 'footer', 'iframe']
        for tag in unuse_tag:
            for s in bs.select(tag):
                if s != None:
                    s.extract()
        return bs

    def clean_html(self, html : str):
        clean = re.compile('<.*?>')
        clean_text = re.sub(clean, '', html)
        for char in ['\n', '\t', '\r']:
            clean_text = clean_text.replace(char, '')
        clean_text = re.sub(' +', ' ', clean_text)
        return clean_text

    def data_in_link(self, session, url):
        self.list_url = []
        self.list_title = []
        self.list_content = []

        with session.get(url) as response:
            if not response.ok:
                return
            print('test url', url)
            
            domain = urlparse(url).netloc
            bs = BeautifulSoup(response.text, 'html.parser')
            bs = self.remove_unuse_tag(bs)
            # print('test bs inlink', bs)
            
            self.data['web'][url] = {}
            self.data['web'][url]['title'] = bs.find('h1').text

            self.content = ''
            section = bs.find('div', {'class' : 'content-area'})
            # print('test sec', section)
            for p in section.find_all('p'):
                # print('test p', p)
                self.content += f'{self.clean_html(p.text)} ' #p.text
            self.data['web'][url]['content'] = self.content
            # print('test content inlink', self.content)



    def data_domain(self, session, url):
        with session.get(url) as response:
            # print('check re', response)
            if not response.ok:
                return
            domain = urlparse(url).netloc
            bs = BeautifulSoup(response.text, 'html.parser')
            bs = self.remove_unuse_tag(bs)
            # print('test bs domain', bs)

            news_link = []
            for h in bs.find_all('h2', {'class' : 'entry-title'}):
                a = h.find('a')
                news_link.append(a.attrs['href'])
            # print('test news_link domain', news_link)

            n = len(news_link)
            # print('test n news_link domain', n)
            with ThreadPoolExecutor(max_workers=n) as executor:
                with requests.Session() as session:
                    executor.map(self.data_in_link, [session]*n, news_link)
                    executor.shutdown(wait=True)


    def scrap(self):
        pages_list = [f'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/{i}' for i in range(1, 11)]
        n = len(pages_list)
        print('test page' , pages_list)
        with ThreadPoolExecutor(max_workers=n) as executor:
            with requests.Session() as session:
                executor.map(self.data_domain, [session]*n, pages_list)
                executor.shutdown(wait=True)


    def save_to_json(self):
        with open(f'Nineentertain.json', 'w', encoding="UTF-8") as file:
            datafile = json.dumps(self.data, indent=4) 
            file.write(datafile)


    # def dataframe_web(self):

    #     data_web = [[self.list_url
    #                     ]]

    #     web_frame = pd.DataFrame(data=data_web,
    #                         columns=['link'])
    #     return web_frame

    # def twitter_to_xlsx(self, data):
    #     data.to_excel("hey_test.xlsx", engine="openpyxl", index=False)


In [10]:
data = testNineentertain()
data

In [11]:
data.scrap()

test page ['https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/1', 'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/2', 'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/3', 'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/4', 'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/5', 'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e0%b8%9b%e0%b8%b4%e0%b8%99%e0%b9%80%e0%b8%84%e0%b8%9b%e0%b9%87%e0%b8%ad%e0%b8%9b/page/6', 'https://nineentertain.mcot.net/tag/%e0%b8%a8%e0%b8%b4%e0%b8%a5%e

In [12]:
# data.dataframe_web()

In [13]:
data.save_to_json()

In [14]:
# hey = data.dataframe_web()
# data.twitter_to_xlsx(hey)